In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install emoji
import emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)

1.15.2


In [0]:
pip install bert-tensorflow

In [0]:
import bert
from keras.optimizers import Adam
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
#mounting drive for the OUTPUT directory

from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/'My Drive'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 501
'545 project'
'545 Project'
 autograder.py
 Capture+_2020-01-30-14-44-34.png
'Colab Notebooks'
'Copy of emotions1_ADD_EMOJI - emotions1.csv'
'CycleGAN 504 Project'
'EECS 501 homework'
'Elsa Mary Mathew - Elsa Mary Mathew.pdf'
'Elsa Mary Mathew.pdf'
 emotions1_ADD_EMOJI.xlsx
 epdf.pub_matrix-analysis-for-scientists-and-engineers.pdf
'Getting started.pdf'
 HW2.pdf
 I-20
 I-20.pdf
'I94 - Official Website.pdf'
 layers.py
 Mathew_ElsaMary_Resume.pdf
 q1.ipynb
 q2.ipynb
 q4.ipynb
' question4.ipynb'
'Sad Tweets #Elsa Don'\''t Edit .gsheet'
 schedule.jpg
 Screenshot_20200128_170406_com.android.chrome.jpg
 TODO
 Untitled1.ipynb
 Untitled.ipynb


In [0]:


# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = '/content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'Checkpoint_Dimple_Elsa' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(OUTPUT_DIR,BUCKET)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: /content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased *****


In [0]:
from tensorflow import keras
import os
import re
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_excel('/content/gdrive/My Drive/emotions1_ADD_EMOJI.xlsx')

In [0]:
df.loc[df['Affect Dimension'] == 'anger','newlabel'] = 0

df.loc[df['Affect Dimension'] == 'joy','newlabel'] = 1

df.loc[df['Affect Dimension'] == 'sadness/fear','newlabel'] = 2

df.loc[df['Affect Dimension'] == 'neutral','newlabel'] = 3
print (df['newlabel'][6034])

1.0


In [0]:
def is_emoji(s):
    count = 0
    for emoji1 in emoji.UNICODE_EMOJI:
        count += s.count(emoji1)
        if count > 1:
            return False
    return bool(count)


def token_emoji_conversion(text):
    words2 = []
    words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    words11=words.copy()
    idx=0
    i2=0
    n=0
    for i1, word in enumerate(words11):
      #print(word)
      if i2!=0 and n!=0:
        words.insert(idx, word1)
        idx+=1
        n=0
      if i2==0 and i1!=0:
        idx+=1
      i2=0
      m=0
      n=0
      for c in word:
      #c for c in str if c in emoji.UNICODE_EMOJI
        if is_emoji(c) == True:
          i2+=1
          if m==0:
            try:
              del(words[idx])
            except:
              print(words, idx)
          if n!=0:
            words.insert(idx, word1)
            idx+=1
            n=0
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', '')
          word1 = nltk.word_tokenize((word1))
          for i in range(len(word1)):
            words.insert(idx, word1[i])
            #print(i)
            idx+=1
          m+=1
        else:
          if n==0:
            word1 = c
          else:
            word1=word1+c
          n+=1
    for token in words:
        if re.search('[a-zA-Z]', token):
            words2.append(token)
    
    #TreebankWordDetokenizer().detokenize(output)
    return TreebankWordDetokenizer().detokenize(words2)

In [0]:
df['Tweet']=df['Tweet'].apply(token_emoji_conversion)


In [0]:
print(df['Tweet'])

0        pouting face chetan_bhagat face with steam fro...
1        Low key Dro f*cked Molly like he had a point t...
2        TeleTransOne eleganza_lily FemFataleXX That's ...
3        F3Alpha F3Roswell Preblast\nI've face with ste...
4        Did they offend us and they want it to sound n...
                               ...                        
12818    You people are fucking mad and I'm here to tel...
12819    god i am so fucking mad at this stupid fucking...
12820    This makes me so mad because everywhere I go w...
12821    Literally I'm so fucking mad it's not even fun...
12822    im gonna commit a fucking crime I m mad poutin...
Name: Tweet, Length: 12823, dtype: object


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['Tweet'], df['newlabel'], test_size=0.2, random_state=42)

print (X_test.head())

7386     r0Ils ppl get triggered over u smiling they're...
10086    Brb going start killing random white people go...
3037     Look at this little guy He is connecting to hi...
3026      na_dirs Omg you're savage prof this is hilarious
2692     My BD is the biggest piece of shit on Earth So...
Name: Tweet, dtype: object


In [0]:
X_test.shape

(2565,)

In [0]:
train = pd.DataFrame({'Tweet':X_train,'newlabel':y_train})
test = pd.DataFrame({'Tweet':X_test,'newlabel':y_test})

DATA_COLUMN = 'Tweet'
LABEL_COLUMN = 'newlabel'
label_list = [0, 1, 2, 3]
label_dict = {'anger':0, 'joy':1, 'sadness/fear':2, 'neutral': 3}


In [0]:




train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
from bert import tokenization

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info rom the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer ")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 10258


INFO:tensorflow:Writing example 0 of 10258


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0.0 (id = 0)


INFO:tensorflow:label: 0.0 (id = 0)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 100

In [0]:

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f396c185978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f396c185978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into /content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into /content/gdrive/My Drive/545 project/21stApril_converting_emoji_to_word_Checkpoint_uncased/model.ckpt.


ResourceExhaustedError: ignored

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

In [0]:
import numpy as np

predictions = estimator.predict(test_input_fn)
print (predictions)
probabilities = []

def logit_tp_prob (logit) :
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return(prob)

for prediction in predictions:
    logits = prediction['probabilities'] 
    p = []
    for logit in logits:
        p.append(logit_tp_prob (logit))
    probabilities.append(p)

np.savetxt("/content/gdrive/My Drive/545 project/20thApril/prob_uncased.csv", probabilities, delimiter=",")

In [0]:
def getPrediction(in_sentences):
  labels = ['anger', 'joy', 'sadness/fear', 'neutral']
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences]
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking🤩",
  "The film was creative and surprising",
  "Absolutely fantastic!🤩"
]

In [0]:
#words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
predictions = getPrediction([token_emoji_conversion(x) for x in pred_sentences])

In [0]:

predictions

In [0]:
#cased_prob = pd.read_csv("/content/gdrive/My Drive/545 project/20thApril/prob_cased.csv",header=None)
#uncased_prob = pd.read_csv("/content/gdrive/My Drive/545 project/20thApril/prob_uncased.csv",header=None)
#cased_prob.columns = ['a','b','c','d']
#cased_prob.shape

In [0]:
#from sklearn.metrics import accuracy_score
#avg_prob = (np.array(cased_prob) + np.array(uncased_prob)) /2
 
#pred_labels = np.argmax(avg_prob, axis = 1)

#accuracy_score(y_test, pred_labels)
